# Лабораторная  работа №6 по курсу "Методы и средства передачи информации"

_Выполнил студент группы А-07-18 Востриков Роман_

## Задание 1

а) Закодировать по методу Хэмминга число 11. В полученный код ввести одиночную ошибку и определить номер искаженного разряда.

б) Пусть $11010011$ и $11001111$ искаженные слова расширенного кода Хемминга длины 8. Какое из этих слов содержит одиночную ошибку, а какое – двойную ошибку? В случае одиночной ошибки определить ее местоположение.

### Решение

### а)
Представим число $11$ в двоичной системе счисления: ${11}_{10} = {1011}_2$.
Сообщение содержит 4 информационных бита, следовательно нужно добавить 3 контрольных бита на позиции 1, 2, 4. Получаем сообщение: $k_1\,k_2\,1\,k_3\,0\,1\,1$, где $k_1$, $k_2$, $k_3$ – контрольные биты.

Найдем контрольные биты:
$$
\begin{array}{lcl}
k_1 = (a_3 + a_5 + a_7)\mod 2 = (1 + 0 + 1)\mod 2 = 0 \\
k_2 = (a_3 + a_6 + a_7)\mod 2 = (1 + 1 + 1)\mod 2 = 1 \\
k_3 = (a_5 + a_6 + a_7)\mod 2 = (0 + 1 + 1)\mod 2 = 0
\end{array}
$$

Итоговое сообщение, закодированное кодом Хэмминга(7,4): $0110011$.

Пусть ошибка будет в 6-м бите, тогда сообщение примет вид: $0110001$. Определим местоположение ошибки, с помощью контрольных битов:
$$
\begin{array}{lcl}
s_1 = (a_1 + a_3 + a_5 + a_7)\mod 2 = (0 + 1 + 0 + 1)\mod 2 = 0\quad & + \\
s_2 = (a_2 + a_3 + a_6 + a_7)\mod 2 = (1 + 1 + 0 + 1)\mod 2 = 1\quad & - \\
s_3 = (a_4 + a_5 + a_6 + a_7)\mod 2 = (0 + 0 + 0 + 1)\mod 2 = 1\quad & -
\end{array}
$$
Контрольные биты, стоящие на позициях 2 и 4 нечётны, отсюда бит ошибки - $2 + 4 = 6$.

### б)
Проверим слово $11010011$:
$$
s = (a_1 + a_2 + a_3 + a_4 + a_5 + a_6 + a_7 + a_8)\mod 2 = (1 + 1 + 0 + 1 + 0 + 0 + 1 + 1)\mod 2 = 1
$$
$s=1$ $\Rightarrow$ в слове допущена одна ошибка. Найдём её.
$$
\begin{array}{lcl}
s_1 = (a_1 + a_3 + a_5 + a_7)\mod 2 = (1 + 0 + 0 + 1)\mod 2 = 0\quad & + \\
s_2 = (a_2 + a_3 + a_6 + a_7)\mod 2 = (1 + 0 + 0 + 1)\mod 2 = 0\quad & + \\
s_3 = (a_4 + a_5 + a_6 + a_7)\mod 2 = (1 + 0 + 0 + 1)\mod 2 = 0\quad & +
\end{array}
$$
$s_1,\, s_2,\, s_3$ равны 0, значит ошибка произошла в бите $a_8$.



Проверим слово $11001111$.
$$
s = (a_1 + a_2 + a_3 + a_4 + a_5 + a_6 + a_7 + a_8)\mod 2 = (1 + 1 + 0 + 0 + 1 + 1 + 1 + 1)\mod 2 = 0
$$
Но при этом:
$$
\begin{array}{lcl}
s_1 = (a_1 + a_3 + a_5 + a_7)\mod 2 = (1 + 0 + 1 + 1)\mod 2 = 1\quad & - \\
s_2 = (a_2 + a_3 + a_6 + a_7)\mod 2 = (1 + 0 + 1 + 1)\mod 2 = 1\quad & - \\
s_3 = (a_4 + a_5 + a_6 + a_7)\mod 2 = (0 + 1 + 1 + 1)\mod 2 = 1\quad & -
\end{array}
$$
Значит в слове допущено 2 ошибки, нельзя определить их местоположение.

## Задание 2

Составить программу для кодирования методом Хемминга двоичного слова длины M и определения  номера  искаженного разряда.

### Решение

Для удобства обработки будем представлять слова в виде массивов двоичных цифр, для преобразования строки в массив и обратно будем использовать вспомогательные функции из лабораторной работы №5.

In [75]:
import numpy as np # импорт библиотеки для облегчения работы с массивами и векторизации вычислений 

In [76]:
# функция для получения массива двоичных цифр из строки
def to_array(msg):
    return [int(d) for d in msg]

# функция для приведения массива двоичных цифр к строковому представлению
def stringify(arr):
    return np.array2string(arr, separator='')[1:-1]

Пред началом реализации алгоритмов кодирования и декодирования определим несколько вспомогательных функций для работы со степенями 2:

In [77]:
# функция проверяет что n > 0 является степенью 2
def is_power_of_2(n):
    return n & (n - 1) == 0

In [78]:
# функция для получения первых n степеней 2
def get_powers_2(n):
    powers = np.arange(n)
    return np.full((n,), 2) ** powers

In [79]:
# определение количества контрольных битов для сообщения длины M
def parity_count(M):
    from math import ceil, floor, log2
    
    n = (M << 1) if is_power_of_2(M) else 2 ** ceil(log2(M)) # n - наименьшая степень 2, превосходящая M
    lower_bin = floor(log2(n))
    upper_bin = lower_bin + 1
    data_bit_boundary = n - lower_bin - 1
    return lower_bin if M <= data_bit_boundary else upper_bin

In [80]:
# определение битов данных, соответствующих контрольному биту в указанной позиции
def data_bits(parity, datalen):
    if is_power_of_2(parity):
        data_index  = 1 # позиция бита в строке данных
        code_index = 3 	# позиция бита в кодовом слове

        indexes = [] # позиции битов в кодовом слове, которые контроллирует данный бит четности 

        while data_index <= datalen:
            curr_bit_is_data = not is_power_of_2(code_index)
            if curr_bit_is_data and (code_index % (parity << 1)) >= parity:
                indexes.append(code_index) 
            data_index += curr_bit_is_data
            code_index += 1

        return np.array(indexes)
    else:
        raise ValueError("Контрольные биты могут находится только в позициях, равных степеням двойки")

Используя определенные выше вспомогательные функции можно реализовать кодирование методом Хемминга:

In [90]:
# функция кодирования простым кодом Хемминга (без бита общей проверки четности)
def encode(msg):
    length = len(msg)
    parity = parity_count(length)
    length += parity # итоговая длина кода
    
    data = np.array(msg) # информационные биты
    powers = get_powers_2(parity) # позиции контрольных битов (считая с 1)
    code = np.zeros(length, dtype = 'i8')
   
    mask = np.full_like(code, True, dtype ='bool') # маска битов данных в коде
    mask[powers-1] = False 
    
    code[mask] = data # устанавливаем биты данных в коде
    
    # устанавливаем контрольные биты
    for p in powers:
        bits = data_bits(p, data.size) - 1 # позиции битов данных(считая с 0)
        code[p-1] = code[bits].sum() % 2
    
    return stringify(code)

In [127]:
# функция декодирования сообщений, закодированных простым кодом Хемминга (без бита общей проверки четности)
# функция возвращает сообщение и позицию ошибки (считая с 1, если ее не было - 0)
def decode(msg):
    from math import floor, log2
    length = len(msg)
    parity =  floor(log2(length)) + 1 # число проверочных битов в кодовом слове
    powers = get_powers_2(parity) # позиции контрольных битов (считая с 1)
    
    code = np.array(msg)
    
    mask = np.full_like(code, True, dtype ='bool') # маска битов данных в коде
    mask[powers-1] = False 
    
    actual = code[np.invert(mask)] # полученные контрольные биты
    
    # вычисляем контролные биты, соответствующие полученному сообщению
    expected = np.zeros_like(actual)
    for i in range(actual.size):
        bits = data_bits(powers[i], length-parity) - 1 # позиции битов данных(считая с 0)
        expected[i] = code[bits].sum() % 2
    
    errors = np.where(expected != actual)[0] # несовпадающие контрольные биты
    
    if errors.size != 0:
        pos = np.sum(2 ** errors) # позиция бита ошибки в кодовом слове (считая с 1)
        code[pos-1] = 0 if code[pos-1] == 1 else 1
        return (stringify(code[mask]), pos)
    else:
        return (stringify(code[mask]), 0)    

Проверим корректность работы кодирования/декодирования на примере из задания 1А, закодируем сообщение 1011:

In [132]:
encode(to_array('1011'))

'0110011'

Мы получили такое же кодовое слово, как и в задании 1, декодируем его, а также версию с одной ошибкой в 6 бите:

In [133]:
print(decode(to_array('0110011')))
print(decode(to_array('0110001')))

('1011', 0)
('1011', 6)


В обоих случаях мы получили верное исходное сообщение, единичная ошибка была обнаружена и исправлена.

## Вывод

В лабораторной работе было показано кодирование сообщений методом Хэмминга. Этот метод удобен тем, что позволяет исправлять единичные ошибки (расширенный код позволяет обнаруживать двойные). Главная особенность этого метода состоит в том, что контрольные биты ставятся на конкретные позиции, а не в конце сообщения, как, например, в прямоугольном коде. 